#By Knowing this notebook, we can become an eligible "Data Egress Developer/Engineer"

In [0]:
ingest_df1 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header", header=True, sep=",",inferSchema=True,samplingRatio=0.10)

###Writing the data in Builtin - different file formats & different targets (all targets in this world we can write the data also...)  
####1. Writing in csv (structured data (2D data Table/Frames with rows and columns)) format with few basic options listed below (Schema (structure) Migration)  
custid,fname,lname,age,profession -> custid~fname~lname~prof~age

- header
- sep
- mode

In [0]:
#We are performing schema migration from comma to tilde delimiter
ingest_df1.write.csv(path="/Volumes/workspace/wd36schema/ingestion_volume/target/csvout", header=True, sep="~", mode="overwrite")
#4 modes of writing - append,overwrite,ignore,error
display(ingest_df1.head)

In [0]:
#We are performing schema migration by applying some transformations (this is our bread and butter that we learn exclusively further)
transformed_df = ingest_df1.select("custid","fname","lname","profession","age").withColumnRenamed("profession","prof")#DSL transformation (not for now...)
transformed_df.write.csv(path="/Volumes/workspace/wd36schema/ingestion_volume/target/csvout", header=True, sep="~", mode="overwrite", compression='gzip')

###2. Writing in json format with few basic options listed below
path  
mode

- We did a schema migration and data conversion from csv to json format (ie structued to semi structured format)
- json - we learn a lot subsequently (nested/hierarchical/complex/multiline...),
- what is json - fundamentally it is a dictionary of dictionaries
- json - java script object notation
- Standard json format (can't be changed) - {"k1":"string value","k2":numbervalue,"k3":v2} where key has to be unique & enclosed in double quotes and value can be anything
- when to go with json or benifits -
a. If we have data in a semistructure format (with variable data format with dynamic schema)  
eg. {"custid":4000001,"profession":"Pilot","age":55,"city":"NY"}
{"custid":4000001,"fname":"Kristina","lname":"Chung","prof":"Pilot","age":"55"}  
b. columns/column names or the types or the order can be different  
c. json will be provided by the sources if the data is dynamic in nature (not sure about number or order of columns) or if the data is api response in nature.  
d. json is a efficient data format (serialized/encoded) for performing data exchange between applications via network & good for parsing also & good for object by object operations (row by row operation in realtime fashion eg. amazon click stream operations)  
e. json can be used to group or create hierarchy of data in a complex or in a nested format eg. https://randomuser.me/api/

In [0]:
ingest_df1.write.json(path="/Volumes/workspace/wd36schema/ingestion_volume/target/csvout/jsonout",mode='append')
#custid,fname,lname,age,profession -> {"custid":4000001,"fname":"Kristina","lname":"Chung","prof":"Pilot","age":55}

####3.Serialization & Deserialization File formats (Binary/Brainy File formats)

####Data mechanics
1. encoding/decoding(machine format) - converting the data from human readable format to machine understandable format for performant data transfer (eg. Network transfer of data will be encoded)
2. *compression/uncompression(encoding+space+time) - shrinking the data in some format using some libraries (tradeoff between time and size) (eg. Compress before store or transfer) - snappy is a good compression tech used in bigdata platform
3. encryption (encoding+security) - Addition to encoding, encryption add security hence data is (performant+secured) (using some algos - SHA/MD5/AES/DES/RSA/DSA..)
4. *Serialization (applicable more for bigdata) - Serialization is encoding + performant by saving space + processing intelligent bigdata format - Fast, Compact, Interoperable, Extensible (additional configs), Scalable (cluster compute operations), Secured (binary format)..
5. *masking - Encoding of data (in some other format not supposed to be machine format) which should not be allowed to decode (used for security purpose)

What are the (builtin) serialized file formats we are going to learn? orc parquet delta(databricks properatory)

- We did a schema migration and data conversion from csv/json to serialized data format (ie s  tructued to sturctured(internall binary unstructured) format)  
- We learn/use a lot/heavily subsequently,  
- what is serialized - fundamentally they are intelligent/encoded/serialized/binary data formats applied with lot of optimization & space reduction strategies..  
- orc - optimized row column format  
- parquet - tiled data format  
- delta(databricks properatory) enriched parquet format - Delta (modified) operations can be performed  
- format - serialized/encoded , we can't see with mere eyes, only some library is used deserialized/decoded data can be accessed as structured data  
- **when to go with serialized or benifits** -  
a. For storage benifits for eg. orc will save 65+% of space for eg. if i store 1gb data it occupy 350 space, with compression it can improved more...  
b. For processing optimization. Orc/parquet/delta will provide the required data alone if you query using Pushdown optimization .  
c. Interoperability feature - this data format can be understandable in multiple environments for eg. bigquery can parse this data.  
d. Secured
**In the projects/environments when to use what fileformats - we learn in detail later

In [0]:
ingest_df1.write.orc(path="/Volumes/workspace/wd36schema/ingestion_volume/target/orcout",mode='overwrite',compression='zlib')#by default orc/parquet uses snappy compression
spark.read.orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout").show(2)#uncompression + deserialization

In [0]:
#Orc/Parquet follows WORM feature (Write Once Read Many)
ingest_df1.write.mode("overwrite").option("compression","gzip").option("compression","snappy").parquet(path="/Volumes/workspace/wd36schema/ingestion_volume/target/parquetout")#by default orc/parquet uses snappy compression
spark.read.parquet("/Volumes/workspace/wd36schema/ingestion_volume/target/parquetout").show(2)#uncompression + deserialization

In [0]:
#Delta follows WMRM feature (Write Many Read Many) - We did Delta Lake creation (Datalake + Delta file format)
ingest_df1.write.format("delta").save("/Volumes/workspace/wd36schema/ingestion_volume/target/deltaout",mode='overwrite')
spark.read.format("delta").load("/Volumes/workspace/wd36schema/ingestion_volume/target/deltaout").show(2)
     

### 4.Table Load Operations - Building LAKEHOUSE ON TOP OF DATALAKE

#####Can we do SQL operations directly on the tables like a database or datawarehouse? or Can we build a Lakehouse in Databricks?

- We learn/use a lot/heavily subsequently,  
- what is Lakehouse - A SQL/Datawarehouse/Query layer on top of the Datalake is called Lakehouse  
- We have different lakehouses which we are going to learn further -  
1. delta tables (lakehouse) in databricks  
2. hive in onprem  
3. bigquery in GCP  
4. synapse in azure  
5. athena in aws  
- when to go with lakehouse  
a. Transformation  
b. Analysis/Analytics  
c. AI/BI  
d. Literally we are going to learn SQL & Advanced SQL  

altering the table in below cell as previously created is having age column as string

In [0]:
%sql
ALTER TABLE workspace.wd36schema.lh_custtbl
CHANGE COLUMN age age INT

In [0]:
%sql
-- Add a new column with INT type
ALTER TABLE workspace.wd36schema.lh_custtbl
ADD COLUMN age_int INT;

-- Update the new column with converted values
UPDATE workspace.wd36schema.lh_custtbl
SET age_int = CAST(age AS INT);

-- Drop the old column
ALTER TABLE workspace.wd36schema.lh_custtbl
DROP COLUMN age;

In [0]:
#We are building delta tables in databricks (we are building hive tables in onprem/we are building bq tables in gcp...)
#saveastable (named notation/named arguments)
#Table
#cid,prof,age,fname,lname
#mapping
#cid,prof,age,fname,lname
ingest_df1.write.saveAsTable("workspace.wd36schema.lh_custtbl",mode='overwrite')
#display(spark.sql("show create table workspace.wd36schema.lh_custtbl"))

In [0]:
#1. insertinto function can be used as like saveAstable with few differences
#a. it works only if the target table exist
#b. it works by creating insert statements in the behind(not bulk load), hence it is slow, hence we have use for small dataset (safely only if table exists)
#c. it will load the data from the dataframe by using position, not by using name..
#insertInto (positional notation/positional arguments)
#Table
#cid,prof,age,fname,lname
#mapping.
#cid,fname,lname,age,prof
ingest_df1.write.insertInto("workspace.wd36schema.lh_custtbl",overwrite=True)

In [0]:
ingest_df1.write.format("delta").save("location")

In [0]:
#I am using spark engine to pull the data from the lakehouse table backed by dbfs (s3) (datalake) where data in delta format(deltalake) 
display(spark.sql("select * from workspace.wd36schema.lh_custtbl"))#sparkengine+lakehouse+datalake(deltalake)
     

####
5. XML Format - Semi structured data format (most of the json features can be applied in xml also, but in DE world not so famous like json)  
- Used rarely on demand (by certain target/source systems eg. mainframes)  
- Can be related with json, but not so much efficient like json  
- Databricks provides xml as a inbuild function

###Modes in Writing  
1.**Append **- Adds the new data to the existing data. It does not overwrite anything.  
2.**Overwrite ** - Replaces the existing data entirely at the destination.  
3. **ErrorIfexist**(default) - Throws an error   if data already exists at the destination.  
4. **Ignore** - Skips the write operation if data already exists at the destination.  

In [0]:
ingest_df1.write.xml("/Volumes/workspace/wd36schema/ingestion_volume/target/xmlout",mode="ignore",rowTag="cust")

What are all the overall options we used in this notebook, for learning fundamental spark dataframe write operations in different formats and targets?

1. df.write.csv/json/orc/parquet/table/xml... operations & df.write.format('delta').save()
2. Few of the important read options under csv such as header, sep, mode(append/overwrite/error/ignore), toDF.
3. Few additional options such as compression, different file formats...
4. Few of the important read options under csv such as header, sep, mode(append/overwrite/error/ignore), toDF.
5. Few additional options such as compression, different file formats...